In [2]:
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U controlnet_aux==0.0.7 
!pip install transformers accelerate safetensors

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/diffusers.git to c:\users\platinum\appdata\local\temp\pip-req-build-572s1civ
  Resolved https://github.com/huggingface/diffusers.git to commit f912f39b50f87e50a9d99346f5c1b6e644653262
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git 'C:\Users\Platinum\AppData\Local\Temp\pip-req-build-572s1civ'


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
from diffusers import StableDiffusionXLAdapterPipeline, T2IAdapter, EulerAncestralDiscreteScheduler, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from controlnet_aux.pidi import PidiNetDetector
import torch

C:\Users\Platinum\anaconda3\envs\ali\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant' (C:\Users\Platinum\anaconda3\envs\ali\lib\site-packages\charset_normalizer\constant.py)

In [6]:
# load adapter
adapter = T2IAdapter.from_pretrained(
  "TencentARC/t2i-adapter-sketch-sdxl-1.0", torch_dtype=torch.float16, varient="fp16"
).to("cpu")

In [8]:

# load euler_a scheduler
model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
euler_a = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
vae=AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    model_id, vae=vae, adapter=adapter, scheduler=euler_a, torch_dtype=torch.float16, variant="fp16", 
).to("cpu")
pipe.enable_xformers_memory_efficient_attention()

pidinet = PidiNetDetector.from_pretrained("lllyasviel/Annotators").to("cpu")




Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]











Fetching 17 files:  12%|█▏        | 2/17 [00:01<00:14,  1.00it/s]


























C:\Users\Platinum\anaconda3\envs\ali\lib\site-packages\huggingface_hub\file_download.py:979: UserWarning: Not enough free disk space to download the file. The expected file size is: 5135.15 MB. The target location C:\Users\Platinum\.cache\huggingface\hub only has 4662.96 MB free disk space.
  warnings.warn(
C:\Users\Platinum\anaconda3\envs\ali\lib\site-packages\huggingface_hub\file_download.py:979: UserWarning: Not enough free disk space to download the file. The expected file size is: 5135.15 MB. The target location C:\Users\Platinum\.cache\huggingface\hub\models--stabilityai--stable-diffusion-xl-base-1.0\blobs only has 4662.96 MB free disk space.
  warnings.warn(













































































Fetching 17 files:  18%|█▊        | 3/17 [33:41<3:16:30, 842.19s/it]













































































































































































































































































































































































































Fetching 17 files:  29%|██▉       | 5/17 [2:00:45<6:00:46, 1803.88s/it]








































































































Fetching 17 files:  82%|████████▏ | 14/17 [3:29:22<44:51, 897.31s/it]  :15<55:11, 436kB/s]


OSError: [Errno 28] No space left on device

In [9]:
url = "https://huggingface.co/Adapter/t2iadapter/resolve/main/figs_SDXLV1.0/org_sketch.png"
image = load_image(url)
image = pidinet(
  image, detect_resolution=1024, image_resolution=1024, apply_filter=True
)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


NameError: name 'pidinet' is not defined

In [ ]:
prompt = "a robot, mount fuji in the background, 4k photo, highly detailed"
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=30,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_sketch.png')